# Introduction

These notebooks are a translation of the Soar tutorials found [here](https://soar.eecs.umich.edu/Downloads) written by John Laird et al. 

The following, however, is aimed to abbreviate the original tutorials. 
For a more in-depth look at Soar, I recommend you read the original tutorials and check out "The Soar Cognitive Architecture" book.

In these tutorials you will learn the basics of writing Soar programs. 

### Basics

Soar knowledge is encoded as *if then* rules, or **productions**.

The first agent we will write will check if it exists and print `hello world`.

**Setup imports**

In [1]:
import sys
sys.path.append('../../src/')

from lib.pysoarlib import *
import os

curdir = os.path.abspath('')

Soar exposes a python binding for ease of use. `pysoarlib` is a package created by `amininger`. You can find it here: https://github.com/amininger/pysoarlib. I have tweaked it a bit to work better in this container environment.

We can create an agent using the SoarAgent class.

In [2]:
rule_agent = SoarAgent(
    agent_name='hello world',
    write_to_stdout=True,
    spawn_debugger=False,
    agent_source=curdir + '/1_hello_world.soar',
)

--------- SOURCING PRODUCTIONS ------------
Total: 1 production sourced.


Next we can add a connector to see the stdout

In [3]:
rule_agent.add_connector('simple', AgentConnector(rule_agent))
rule_agent.connect()
rule_agent.execute_command('run')

Hello World
System halted.
Interrupt received.This Agent halted.


'--> 1 decision cycle executed. 1 rule fired. \nAn agent halted during the run.'

## Coding the Agent

The pseudocode of our agent is as follows:
```
If I exist, then write "Hello World" and halt.
```

Easy enough. 
Translating this to Soar will seem counter-intuitive at first, but after a few examples, you will get the hang of the syntax.

In Soar, we write *productions* which are *if then* rules in essence.
These rules are how we program Soar agents.

`If <condition> Then <do action>`

A Soar agent will check to see if the *condition* matches it's internal state of the world (it's **working memory**).

Upon initialization, Soar will be populated with a few working memory **elements**. We can check to see if one of these *elements* is in *working memory*, thus creating the "If I exist" condition.

`If I exist, then write "Hello World" and halt.` written as a Soar rule looks like this:

In [4]:
hello_world_rule = """
sp {hello-world
    (state <s> ^type state)
-->
    (write |Hello World|) 
    (halt)}
"""

### Breaking Down the Rule

![hello world breakdown](img/1_hello-world-breakdown.png)

Soar rules follow this generic template of `if <condition> then <action>`.

```
sp {rule*name
   (condition)
   (condition)
   (condition)
   ...
-->
   (action)
   (action)
   ....}
```

The rule name can be almost anything combination of numbers, letters, and dashes. 
The exception to this is that they cannot be named as a single letter followed by a number (`O3`, `S1`) because Soar reserves these for it's own use.
But there's no reason you should write a rule name like that, rule names should be descriptive.

#### Conditions

Soar stores state in a few different ways. 
Right now, we will only focus on working memory. 
Soar can also store memories in a more persistent manner using its **episodic** or **semantic** memory modules.

All conditions do is check whether or not some item is in (or absent from) working memory.
Every condition in a rule has to be true in order for the rule to *fire*.

#### Actions

When rules fire, every action in that rule will be performed.
Right now, the only action we're working with is printing to screen and halting.
Later, we'll create actions that propose **operators**.
We will use *operators* as a way to present different options as to what a Soar agent *can* do in a given state.
We will then use **preferences** to denote what a Soar agent *should* do in a given state.
More on this later.

Most actions, will update the agent's working memory.

## Working Memory

Working memory is where short-term information is stored. 
It can contain anything that will be useful for your agent to accomplish a task, like the position of nearby objects, sensor data, the health of an enemy, current goals, etc.
This information is scopped *globally* (like a global variable in Python or Java).

Take, for example, the toy problem of stacking blocks. 
Our initial state is two blocks, `A` and `B`, stacked on a `table`.

![Blocks](img/1_stacked-blocks.png)

The state of this problem has three items: `Block 1`, `Block 2`, `Table 1`.
`Block 1` is `ontop` of `Block 2`. `Block 2` is `ontop` of `Table 1`. 
`Block 1` is `blue` and has the name `A`, etc.

We can represent this information in a graph structure.
This is exactly how Soar represents information in working memory.

![Graph](img/1_stacked-blocks-graph.png)

Working memory consists of **elements** which are triples: 

(**indentifier**, **attribute**, **value**)

**Indentifiers** are like nodes on a graph. 
They can have vertices connecting them to other nodes in the graph. 
These vertices are called **attributes** in Soar. 
They point to **values**.

**Values** can be constants, like the string "A".

(`B1`, `^name`, `A`)

Or, **values** can themselves be **identifiers**.

(`State 1`, `^block`, `B1`)

A value that is a indentifier, can point to other identifiers. Constant values cannot.

The problem you are trying to solve will dictate the exact structure of your working memory graph.
You'll have the flexibility to design it however you want.

Every Soar agent is initialized with the following working memory graph.

![Initialized graph](img/1_initialized-wm-graph.png)

`S1` is the initial Soar state. You can see three attributes on it, `io` (input output), `type`, and `superstate`. 
Since `S1` is the only state, there is no `superstate`. 
Nested states (called `substates` in Soar) are covered in later.

You can ignore the `^io` attribute for now. 
We will cover that in the third tutorial notebook.

#### Objects and Augmentations

**Object** → Collection of working memory elements that share the same first identifier.

> "A working memory object usually represents something about the world, such as a block, a wall, a piece of food, or a cell on the board. The individual augmentations represent properties (such as color, size, or weight), or relations with other objects (such as on top of, behind, or inside)."

**Augmentation** → The working memory elements that make up an object.

We can write objects as a list of all of its augmentations.
The first item in the list is the identifier that all augmentations emanate from.

```
(S1 ^io I1 ^superstate nil ^type state)
(I1 ^input-link I3 ^output-link I2)
```

## Back to Hello World

```
If I exist, then write "Hello World" and halt.
```

We can use the fact that Soar agents will be initialized with the state `s1` to check if our agent "exists". 
Our condition will check if an *indentifier* with the *attribute* `^type` exists in working memory, where the *value* of `^type` is `state`. 

> The obvious condition to write would be `(s1 ^type state)`. 
However, `s1` is just an arbitrary symbol and might not be the identifier of that state every time the agent is run. 
Thus,we need a test that there is an identifier, but without testing a specific value. 
That is exactly what a variable does – it matches any symbol in working memory with the only constraint being that all occurrences of the same variable in a rule match the same symbol. 


A variable can match an identifier, an attribute, or a value. 

```
<variable-identifier> ^attribute value
identifier ^<variable-attribute> value
identifier ^attribute <variable-value>
```

This is why our condition looked like this: `state <s> ^type state`.

`<s>` is the variable. 
It could be named whatever. 
It was chosen as `s` because it is a state, but I could've called it `some-state`. 
The important part is that it is surrounded by `<>`.

![Breakdown](img/1_hello-world-breakdown-2.png)

Below is the full Soar program again.

This time, we load a config file which will contain the settings:
```
spawn-debugger = false
write-to-stdout = true
# How much detail to print when sourcing files: none, summary, or full
source-output = summary
```

Instead of giving `SoarAgent` a path to a Soar file, we will load the Soar program from a string (using `agent_raw`).

In [5]:
hello_world_rule = """
sp {hello-world
    (state <s> ^type state)
-->
    (write |Hello World!!|) 
    (halt)}
"""

my_agent = SoarAgent(
    agent_raw=hello_world_rule,
    config_filename='default.config',
    root_dir=curdir,
)

my_agent.add_connector("hw-agent", AgentConnector(my_agent))
my_agent.connect()
result = my_agent.execute_command("run")

print(result)

--------- SOURCING PRODUCTIONS ------------
Total: 1 production sourced.
Hello World!!
System halted.
Interrupt received.This Agent halted.
--> 1 decision cycle executed. 1 rule fired. 
An agent halted during the run.


## Operators

**Operators** will be our mechanism of performing actions.
Actions can be performed by modifying the actual world (moving a block from one location to another) or modify internal state ("thinking" about moving a block from one location to the other).

Examples of operator usage: remembering a health pack to pick up later, moving to a room, turning, keeping track of battery level, etc.

We will use operators to shape the choices a Soar agent makes. 
Operators are managed by production rules and stored in working memory.

The operator production rules will
1. **Propose** operators that match a state (aka operator proposal rules).
2. **Select** an operator and store it in working memory (operator selection rules).
3. **Apply** that operator (operator application rules).

The Soar agent will find all of the operator rules that match the current state and **propose** them.
These are all of the rules that the agent *can* do. 
From these *proposals*, the agent then will **select** what it *should* do.
Finally, there will be rules that check for a given operator in working memory and then take some action (print, halt, tell the agent to move, etc.)

For example, a Mars rover agent *could* move forward, left, right, or backwards.
This agent would propose four operators for doing so.
If it's goal is to it's left, it *should* move in that direction. 
Therefore, it will *select* that direction.
A working memory element will be created that stores the operator `move` and `^direction` `left`.
Finally, that operator will be *applied* by another rule.

This is how decision making happens in Soar.

![Decision cycle](img/1_simple-decision-cycle.png)

How do we weight the *should* criteria on an operator? We use **preferences**. Preferences will be covered in more detail later.

### Hello World Operator

Let's break up the above hello world rule into two rules: a rule that will *propose* the `hello-world` *operator* and a rule that will *apply* a `hello-world` *operator*.

In English, we'd write that rule as,
```
Propose*hello-world:
If I exist, 
propose the hello-world operator.

Apply*hello-world:
If the hello-world operator is selected, 
write “Hello World” and halt.
```

To write the first rule in Soar, we can take the `If I exist` that we wrote earlier as the condition.
The action will be to propose an operator.

(It took me some time to fully wrap my head around the following syntax. If you're confused, I recommend to skim ahead a bit to see other examples. Then, come back to this point and re-read these sections in more depth.)

![Propose hello world](img/1_propose-hello-world.png)

To say that we *can* do something in Soar, we say that we *prefer* as *acceptable*. 
Later, you'll learn how to denote that one operator is better or worse than another.
This is also handled with the concept of *preferences*; this shapes what a Soar agent *should* do.

> The value of the preference, `<o>`, is a new variable that did not occur in the condition. 
When new variables appear in actions, Soar automatically creates a new identifier and uses it for all occurrences of that variable in the action. 
For example, if `o1` is the identifier created for `<o>`, then `(s1 ^operator o1 +)` and `(o1 ^name hello-world)` are added to working memory. 
For variables in actions, such as `<o>`, a different identifier is created each time a rule fires.

In this case, Soar does not have to deliberate over which operator it should choose, as there is only one proposed operator.
After this rule is proposed, it will be selected during the decision procedure. 

Next, the rule to apply a `hello-world` operator will fire.

![Hello world application](img/1_hello-world-application.png)

The action of this rule is the exact same as the first rule we wrote.
The new part here is the condition, `check if there is a state, <s>, that has a link, ^operator, pointing to an identifier, <o>, with the name "hello-world"`.

In [6]:
hello_world_operator_rules = """
sp {propose*hello-world
   (state <s> ^type state)
-->
   (<s> ^operator <o> +)
   (<o> ^name hello-world)
}

sp {apply*hello-world
   (state <s> ^operator <o>)
   (<o> ^name hello-world)
-->
   (write |Hello World|)
   (halt)
}
"""
    
hwo_agent = SoarAgent(
    agent_raw=hello_world_operator_rules,
    config_filename=curdir + '/default.config',
)

hwo_agent.execute_command('run');

--------- SOURCING PRODUCTIONS ------------
Total: 2 productions sourced.


#### Examining Working Memory

With `print`, we can view all the attributes and values that have `s1` as their identifier.

In [7]:
hwo_agent.execute_command("print s1", print_res=True);

print s1
(S1 ^epmem E1 ^io I1 ^operator O1 ^operator O1 + ^reward-link R1 ^smem L1
       ^superstate nil ^type state)


The attributes `^io`, `^superstate`, and `^type` are created automatically before the program runs.
The 'operator' attribute is created when the hello-world operator (o1) is selected.

Printing at a depth of 2 (we abbreviate `print` to `p`) gives us...

In [8]:
hwo_agent.execute_command("p s1 -d 2", print_res=True);

p s1 -d 2
(S1 ^epmem E1 ^io I1 ^operator O1 ^operator O1 + ^reward-link R1 ^smem L1
       ^superstate nil ^type state)
  (E1 ^command C1 ^present-id 1 ^result R2)
  (I1 ^input-link I2 ^output-link I3)
  (O1 ^name hello-world)
  (L1 ^command C2 ^result R3)


#### View the augmentation of `o1`

(Remember that augmentation refers to "the working memory elements that make up an object.")

In [9]:
hwo_agent.execute_command("p o1", print_res=True);

p o1
(O1 ^name hello-world)


####  View the augmentation of `I1`

We've glossed over the `I1` identifier up to this point but we can get a sense of what it does by checking out it's augmentations.

In [10]:
hwo_agent.execute_command("p I1", print_res=True);

p I1
(I1 ^input-link I2 ^output-link I3)


There are two attributes: `^input-link` and `^output-link`

`input-link`: where an agent's sensory information is available in WM

`output-link`: where action commands must be created for the agent to move in its world

We will use these in the third tutorial.